In [1]:
import tracemalloc

from utils_v1 import *
import traceback


In [2]:
from icecream import ic

import numpy as np
from src.dataset import (
    MultipleDates, PAMultipleDates,
    MTMultipleDates
)

# from src.dataset_legacy import MTDeforestationTime
from src import training_generator as generator
from src.patchesHandler import PatchesHandler, PatchesHandlerMultipleDates, PatchesHandlerEvidential

from src.manager.evidential import ManagerEvidential
from src.manager.multioutput import ManagerMCDropout, ManagerSingleRun, ManagerEvidential2
from mpl_toolkits.axes_grid1 import make_axes_locatable

import cv2
from src import metrics as _metric
import json
import pickle 
import src.manager.evidential as evidential
from src.Logger import Logger
import pdb


Tensorflow ver. 2.6.0


In [3]:

with open('config.json', 'r') as f:
    config = json.load(f)
    
print("Mode: {}. Site: {}. Training: {}. Training date: {}".format(
    config['mode'], config['site'], config['training'], config['training_date']))
print(config)


Mode: evidential2. Site: PA. Training: True. Training date: current
{'training': True, 'inferring': True, 'site': 'PA', 'training_date': 'current', 'mode': 'evidential2', 'training_times': 10, 'learning_rate': 0.0001, 'weights': [0.1, 0.9, 0], 'save_probabilities': False, 'loadInference': False, 'addPastDeforestationInput': True, 'Uncertainty': {}, 'uncertainty_method': 'pred_entropy'}


In [4]:


#dates = [2018, 2019]

if config['site'] == 'PA':
	if config['training_date'] == "earlier":
		dates = [2017, 2018]
	else:
		dates = [2018, 2019]

	dataset = PAMultipleDates(config = config,
		dates = dates, # dates = [2016,2017,2018,2019]
		addPastDeforestationInput = True,
		borderBuffer = 2)


	if config['mode'] == 'ensemble':
		if config['training'] == True:
			config['mode'] = 'single_run'
		else:
			sys.exit("Ensure that training is True for ensemble mode. Use infer_ensemble.ipynb for inference.")

	if config['mode'] == 'mcd' or config['mode'] == 'single_run': # MCD will do 10 inference on each of the 10 repetitions = 100 runs
		if config['training_date'] == 'current': exp = 0
			# exp_ids = [103, 104, 105, 106, 107, 118, 119, 120, 121, 122]
			
		elif config['training_date'] == 'earlier': exp = 1
			# exp_ids = [123, 124, 125, 126, 127, 128, 129, 131, 1122, 1121]
			
	if config['mode'] == 'evidential2':
		if config['training_date'] == 'current': exp = 17; repetition_id = 0
		elif config['training_date'] == 'earlier': exp = 6; repetition_id = 0

else:
	if config['training_date'] == "earlier":
		dates = [2018, 2019]
	else:
		dates = [2019, 2020]

	dataset = MTMultipleDates(config = config,
		dates = dates, # dates = [2016,2017,2018,2019]
		addPastDeforestationInput = True,
		borderBuffer = 2)	
	if config['mode'] == 'evidential':	

		# MT
		# exp_id_base = 196
		# early date
		exp_id_base = 206
		# early date inference
		exp_id_base = 210
		exp_id_base = 210 if config['training_date'] == 'earlier' else 206

	elif config['mode'] == 'mcd' or config['mode'] == 'single_run':
		if config['training_date'] == 'current': exp = 2
#			exp_ids = [108, 109, 110, 111, 112, 113, 114, 115, 116, 117]
		elif config['training_date'] == 'earlier': exp = 3
#			exp_ids = [130, 132, 133, 134, 135, 136, 137, 1125, 1124, 1123]

ic(dates)


if issubclass(type(dataset), MultipleDates):
	if config['mode'] == 'evidential':
		patchesHandler = PatchesHandlerEvidential(
			dataset)
	else:
		patchesHandler = PatchesHandlerMultipleDates(
			dataset)
		
else:
	patchesHandler = PatchesHandler()



ic| list(self.date_ids): [0]
ic| self.image_channels: [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]]
ic| dates: [2018, 2019]
ic| self.input_image_shape: 21
ic| self.dataset.image_channels: [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]]


In [5]:


if config['mode'] == 'evidential2':
    config["inference_times"] = 1
    manager_class = ManagerEvidential2
    config['uncertainty_method'] = "pred_entropy"
    config['classes_mode'] = True
    
if config['mode'] == 'mcd':
    config["inference_times"] = 10
    manager_class = ManagerMCDropout
    config['uncertainty_method'] = "pred_entropy"
    config['classes_mode'] = False

elif config['mode'] == 'single_run':
    config["inference_times"] = 1
    manager_class = ManagerSingleRun
    config['uncertainty_method'] = "pred_entropy_single"
    config['classes_mode'] = False



In [6]:
exp

17

In [ ]:
if config['training'] == True:
    idx = 0
    error_count = 0
    
    while idx < config["training_times"]:
        try:

            print("Beginning run number {}".format(idx))
            logger = Logger()
            manager = manager_class(config, dataset, patchesHandler, logger)
            print("manager.config['dropout_training']", manager.config['dropout_training'])
            manager.defineExperiment(exp) # fixed
            manager.defineRepetitionId(idx) # varying from 0 to 10
            
            manager.setExperimentPath()
            manager.createLogFolders()

            manager.loadDataset()

            # %%
            if config["training"] == True:
                manager.run()
            idx += 1
            print("Grid execution idx: {}".format(idx))            
        except Exception as e: 
            print(traceback.format_exc())
            error_count += 1
            if error_count > 10:
                sys.exit()
            print("Execution error in idx {}. Retrying...".format(idx))
            continue


Beginning run number 0
manager.config['dropout_training'] False
D:/Jorge/datasets/deforestation/PA/deforestation_past_years.tif
Loaded deforestation past years


ic| np.unique(deforestation_past_years, return_counts=True): (array([   0, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017,
                                                                    2018, 2019, 2020, 2021], dtype=uint16),
                                                              array([132185553,   3150503,   4571354,    864429,    903608,   1299506,
                                                                      2477971,   1561686,   1459696,    969060,    644976,   2209014,
                                                                      2100204,   5257014,   3514616], dtype=int64))


Label where deforestation past years is actual date (2019) = 1


ic| np.unique(deforestation_past_years, return_counts=True): (array([   0, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017,
                                                                    2018, 2019, 2020, 2021], dtype=uint16),
                                                              array([132185553,   3150503,   4571354,    864429,    903608,   1299506,
                                                                      2477971,   1561686,   1459696,    969060,    644976,   2209014,
                                                                      2100204,   5257014,   3514616], dtype=int64))


Past deforestation different from 0 (no deforestation)


...

Past deforestation before 2008 is 2
D:/Jorge/datasets/deforestation/PA/deforestation_before_2008_PA.tif


ic| np.unique(label_past_deforestation_before_2008, return_counts=True): (array([  0, 215], dtype=uint8), array([124211220,  38957970], dtype=int64))
ic| np.unique(deforestation_past_years, return_counts=True): (array([   0, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017,
                                                                    2018, 2019, 2020, 2021], dtype=uint16),
                                                              array([132185553,   3150503,   4571354,    864429,    903608,   1299506,
                                                                      2477971,   1561686,   1459696,    969060,    644976,   2209014,
                                                                      2100204,   5257014,   3514616], dtype=int64))


Traceback (most recent call last):
  File "C:\Users\jchamorro\AppData\Local\Temp\ipykernel_19660\1756826738.py", line 18, in <cell line: 1>
    manager.loadDataset()
  File "d:\Jorge\deforestation_uncertainty\src\manager\base.py", line 168, in loadDataset
    self.loadLabel()
  File "d:\Jorge\deforestation_uncertainty\src\manager\base.py", line 74, in loadLabel
    self.label_mask = self.dataset.loadLabel()
  File "d:\Jorge\deforestation_uncertainty\src\dataset.py", line 471, in loadLabel
    if self.config['use_cloud_mask'] == True:
KeyError: 'use_cloud_mask'

Execution error in idx 0. Retrying...
Beginning run number 0
manager.config['dropout_training'] False
D:/Jorge/datasets/deforestation/PA/deforestation_past_years.tif
Loaded deforestation past years


ic| np.unique(deforestation_past_years, return_counts=True): (array([   0, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017,
                                                                    2018, 2019, 2020, 2021], dtype=uint16),
                                                              array([132185553,   3150503,   4571354,    864429,    903608,   1299506,
                                                                      2477971,   1561686,   1459696,    969060,    644976,   2209014,
                                                                      2100204,   5257014,   3514616], dtype=int64))


Label where deforestation past years is actual date (2019) = 1


In [ ]:

if config['site'] == 'PA':

	dates = [2018, 2019]

	dataset = PAMultipleDates(dates = dates, # dates = [2016,2017,2018,2019]
		addPastDeforestationInput = True,
		borderBuffer = 2)	

elif config['site'] == 'MT':

	dates = [2019, 2020]

	dataset = MTMultipleDates(dates = dates, # dates = [2016,2017,2018,2019]
		addPastDeforestationInput = True,
		borderBuffer = 2)	


ic(dates)

if issubclass(type(dataset), MultipleDates):
	if config['mode'] == 'evidential':
		patchesHandler = PatchesHandlerEvidential(
			dataset)
	else:
		patchesHandler = PatchesHandlerMultipleDates(
			dataset)
		
else:
	patchesHandler = PatchesHandler()



ic| list(self.date_ids): [0]
ic| self.image_channels: [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]]
ic| dates: [2018, 2019]
ic| self.input_image_shape: 21
ic| self.dataset.image_channels: [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]]


In [ ]:
manager_class

src.trainer.multioutput.TrainerSingleRun

In [ ]:
repetition_ids = manager_class(config, dataset, patchesHandler, logger).get_repetition_ids(exp)  
repetition_n = len(repetition_ids)
print("repetition_ids {}, repetition_n {}".format(repetition_ids, repetition_n))


In [ ]:
if config['inferring'] == True:
    idx = 0
    results = []
    error_count = 0

    while idx < len(repetition_n):
    # for experiment_value, exp_id in zip(experiment_values, exp_ids):
        try:

            print("Beginning run number {}".format(idx))
            logger = Logger()
            manager = manager_class(config, dataset, patchesHandler, logger)
            print("manager.config", manager.config)

            manager.defineExperiment(exp) # fixed
            manager.defineRepetitionId(idx) # varying from 0 to 10
            manager.setExperimentPath()
            manager.createLogFolders()

            manager.loadDataset()

            # %%
            result = manager.run_predictor()
            print("Result idx {}: {}".format(idx, result))
            results.append(result)


            with open(manager.default_log_name, 'wb') as f:
                pickle.dump(results, f)
            print("Grid execution idx: {}".format(idx))
            idx += 1
            print("Grid execution idx: {}".format(idx))
            error_count = 0
        except Exception as e: 
            print(traceback.format_exc())
            error_count += 1
            if error_count > 10:
                sys.exit()
            print("Execution error in idx {}. Retrying...".format(idx))
            continue

Beginning run number 0
trainer.config {'training': False, 'inferring': True, 'site': 'PA', 'training_date': 'current', 'mode': 'single_run', 'learning_rate': 0.0001, 'weights': [0.1, 0.9, 0], 'save_probabilities': False, 'loadInference': False, 'addPastDeforestationInput': True, 'Uncertainty': {}, 'uncertainty_method': 'pred_entropy_single', 'inference_times': 1, 'classes_mode': False, 'dropout_training': False}
D:/Jorge/datasets/deforestation/PA/deforestation_past_years.tif
Loaded deforestation past years


ic| np.unique(deforestation_past_years, return_counts=True): (array([   0, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017,
                                                                    2018, 2019, 2020, 2021], dtype=uint16),
                                                              array([132185553,   3150503,   4571354,    864429,    903608,   1299506,
                                                                      2477971,   1561686,   1459696,    969060,    644976,   2209014,
                                                                      2100204,   5257014,   3514616], dtype=int64))


Label where deforestation past years is actual date (2019) = 1


ic| np.unique(deforestation_past_years, return_counts=True): (array([   0, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017,
                                                                    2018, 2019, 2020, 2021], dtype=uint16),
                                                              array([132185553,   3150503,   4571354,    864429,    903608,   1299506,
                                                                      2477971,   1561686,   1459696,    969060,    644976,   2209014,
                                                                      2100204,   5257014,   3514616], dtype=int64))


Past deforestation different from 0 (no deforestation)


...

Past deforestation before 2008 is 2
D:/Jorge/datasets/deforestation/deforestation_before_2008/deforestation_before_2008_para.tif


ic| np.unique(label_past_deforestation_before_2008, return_counts=True): (array([  0, 215], dtype=uint8), array([124211220,  38957970], dtype=int64))
ic| np.unique(deforestation_past_years, return_counts=True): (array([   0, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017,
                                                                    2018, 2019, 2020, 2021], dtype=uint16),
                                                              array([132185553,   3150503,   4571354,    864429,    903608,   1299506,
                                                                      2477971,   1561686,   1459696,    969060,    644976,   2209014,
                                                                      2100204,   5257014,   3514616], dtype=int64))
ic| label_per_date.shape: (17730, 9203, 1)


(array([0., 1.], dtype=float32), array([162534673,    581327], dtype=int64))
[0. 1.]


ic| pred_entropy.shape: (17760, 9216)
ic| self.label_mask.shape: (17730, 9200)
ic| self.mask_amazon_ts.shape: (17730, 9200)
ic| self.label_mask_test.shape: (81558000,)
ic| self.error_mask_test.shape: (54135604,)
ic| trainer.dataset.previewLims1: array([2200, 3200, 6900, 7900])
    trainer.dataset.previewLims2: array([ 500, 1500, 3500, 4500])
ic| np.unique(trainer.mask_amazon_ts[lims[0]:lims[1], lims[2]:lims[3]]): array([1], dtype=uint8)
ic| np.unique(trainer.mask_amazon_ts[lims[0]:lims[1], lims[2]:lims[3]], return_counts=True): (array([1], dtype=uint8), array([1000000], dtype=int64))
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


(54135604,) (54135604,)
0.8275843097139371
ECE. Starting bin 0...
ECE. Starting bin 1...
ECE. Starting bin 2...
ECE. Starting bin 3...
ECE. Starting bin 4...


In [ ]:
error_count

0

In [ ]:
print(results)

[{'other_uncertainty_metrics': {'sUEO': 0.7995724287077376, 'ece_score': 0.004703061421809009}, 'uncertainty_result': {'metrics': {'precision_L': array([0.75129831]), 'recall_L': array([0.97057467]), 'recall_Ltotal': array([0.61832539]), 'AA': array([0.02999947]), 'precision_H': array([0.65686753]), 'recall_H': array([0.67027312]), 'UEO': array([0.13832113]), 'f1_L': array([0.84697433]), 'f1_H': array([0.66350262]), 'f1': 78.56}, 'metrics_audited': {'precision': array([0.82740766]), 'recall': array([0.98125396]), 'f1': array([0.89778766])}, 'exp': 108}}, {'other_uncertainty_metrics': {'sUEO': 0.8040191984323906, 'ece_score': 0.004066924262362896}, 'uncertainty_result': {'metrics': {'precision_L': array([0.80184449]), 'recall_L': array([0.9597654]), 'recall_Ltotal': array([0.66484832]), 'AA': array([0.02999988]), 'precision_H': array([0.54334729]), 'recall_H': array([0.61293739]), 'UEO': array([0.15867725]), 'f1_L': array([0.87372647]), 'f1_H': array([0.57604823]), 'f1': 78.43}, 'metric

In [ ]:
print(len(results))

10


In [ ]:
results[0]

{'other_uncertainty_metrics': {'sUEO': 0.7995724287077376,
  'ece_score': 0.004703061421809009},
 'uncertainty_result': {'metrics': {'precision_L': array([0.75129831]),
   'recall_L': array([0.97057467]),
   'recall_Ltotal': array([0.61832539]),
   'AA': array([0.02999947]),
   'precision_H': array([0.65686753]),
   'recall_H': array([0.67027312]),
   'UEO': array([0.13832113]),
   'f1_L': array([0.84697433]),
   'f1_H': array([0.66350262]),
   'f1': 78.56},
  'metrics_audited': {'precision': array([0.82740766]),
   'recall': array([0.98125396]),
   'f1': array([0.89778766])},
  'exp': 108}}

In [ ]:
manager.f1

81.35

In [ ]:
print(manager.f1)

81.35
